In [1]:
import numpy
import binascii
import cv2, base64
import numpy as np
from pprint import pprint

In [2]:
def decode(enc, key):
    dec = []
    enc = base64.urlsafe_b64decode(enc).decode()
    for i in range(len(enc)):
        key_c = key[i % len(key)]
        dec_c = chr((256 + ord(enc[i]) - ord(key_c)) % 256)
        dec.append(dec_c)
    dec = "".join(dec)
    return dec

def bits2text(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    Bits2text = n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'
    return Bits2text

In [3]:
class Texty:
    def __init__(self, filename, n=2):
        self.filename = filename        
        self.image = cv2.imread(self.filename)
        self.n = n
    
    decrypt = lambda self, text, key: decode(text, key)
    bits2text = lambda self, bits: bits2text(bits)
    int2bin = lambda self, x: format(x, 'b')
    bin2int = lambda self, x: int(format(int(x, 2), 'd'))
    extractItemsBack = lambda self, num: self.int2bin(num)[-n:]    
    
    def loadImage(self):
        length, count, string = 12, 0, ''
        dataSaver, image = [], self.image
        for i in range(len(image)):
            for j in range(len(image[i])):
                for k in range(3):
                    if count>= length: break
                    count += 1
                    element = image[i][j][k]            
                    element = self.extractItemsBack(element)
                    dataSaver.append(element)
                if count>= length: break
            if count>= length: break

        self.bin24 = ''.join(dataSaver)
        self.distance = self.bin2int(self.bin24)
        
    def changeImage(self):
        
        length, count, string = self.distance+12, 0, ''
        dataSaver = []    
        image = self.image
        for i in range(len(image)):
            for j in range(len(image[i])):
                for k in range(3):
                    if count>= length: break
                    count += 1
                    element = image[i][j][k]            
                    element = self.extractItemsBack(element)
                    dataSaver.append(element)
                if count>= length: break
            if count>= length: break

        self.bin24 = ''.join(dataSaver)[24:]
    
    def getText(self, key):
        encryptedText = self.bits2text(self.bin24)
        return self.decrypt(encryptedText, key)

In [9]:
a = Texty('changed-abc.png', n=2)

a.loadImage()
a.changeImage()

In [10]:
a.getText('password24')

'Text messaging, or texting, is the act of composing and sending electronic messages, typically consisting of alphabetic and numeric characters, between two or more users of mobile phones, tablets, desktops/laptops, or other devices. Text messages may be sent over a cellular network, or may also be sent via an Internet connection.\n\nThe term originally referred to messages sent using the Short Message Service (SMS). It has grown beyond alphanumeric text to include multimedia messages (known as MMS) containing digital images, videos, and sound content, as well as ideograms known as emoji (happy faces, sad faces, and other icons).\n\nAs of 2017, text messages are used by youth and adults for personal, family and social purposes and in business. Governmental and non-governmental organizations use text messaging for communication between colleagues. As with emailing, in the 2010s, the sending of short informal messages has become an accepted part of many cultures.[1] This makes texting a 